# Automated ML

Below are the dependencies that will be needed to complete the project.

In [17]:
import azureml.core
from azureml.core.dataset import Dataset
from azureml.core.workspace import Workspace
from azureml.core import Experiment, Webservice, Model
from azureml.train.automl import AutoMLConfig
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice

import pandas as pd
import sklearn
import json

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.19.0


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [18]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.subscription_id, sep = '\n')

quick-starts-ws-132148
aml-quickstarts-132148
cdbe0b43-92a0-4715-838a-f2648cc7ad21


In [19]:
# choose a name for experiment
experiment_name = 'capstone-heart-failure-exp'

experiment=Experiment(ws, experiment_name)
experiment

Name,Workspace,Report Page,Docs Page
capstone-heart-failure-exp,quick-starts-ws-132148,Link to Azure Machine Learning studio,Link to Documentation


In [20]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException


# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
cluster_name="cpu-cluster"

try:
    new_cluster=ComputeTarget(workspace=ws, name="cpu-cluster")
    print("Existing cluster detected, make use of it!")
    
except ComputeTargetException:
    print("New compute cluster creation in progress...")
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',vm_priority='dedicated',min_nodes=None,max_nodes=4)
    new_cluster = ComputeTarget.create(ws, "new_cluster", compute_config)    
    new_cluster.wait_for_completion(show_output=True)         
    print("Cluster is ready")

Existing cluster detected, make use of it!


In [21]:
datastore = ws.get_default_datastore()

datastore.upload(src_dir='./', target_path='dataset')

Uploading an estimated of 17 files
Target already exists. Skipping upload for dataset/automl.ipynb
Target already exists. Skipping upload for dataset/.ipynb_aml_checkpoints/automl-checkpoint2020-11-28-12-0-54.ipynb
Target already exists. Skipping upload for dataset/.ipynb_checkpoints/automl-checkpoint.ipynb
Target already exists. Skipping upload for dataset/classification-credit-card-fraud/auto-ml-classification-credit-card-fraud.ipynb
Target already exists. Skipping upload for dataset/classification-credit-card-fraud/auto-ml-classification-credit-card-fraud.yml
Target already exists. Skipping upload for dataset/classification-credit-card-fraud/.ipynb_checkpoints/auto-ml-classification-credit-card-fraud-checkpoint.ipynb
Target already exists. Skipping upload for dataset/deploy-to-cloud/model-register-and-deploy.ipynb
Target already exists. Skipping upload for dataset/deploy-to-cloud/model-register-and-deploy.yml
Target already exists. Skipping upload for dataset/deploy-to-cloud/README.

$AZUREML_DATAREFERENCE_4e647ff9a9f248d0852fc6848fb58af8

In [ ]:
dataset = Dataset.get_by_name(ws, name="heart-failure")
print("Dataset is registered")
dataset

In [ ]:
dataset.take(5).to_pandas_dataframe()

## AutoML Configuration

The following code shows a basic example of creating an AutoMLConfig object and submitting an experiment for classification.
I chose the automl settings below because I wanted to specify the experiment type as classification. The classification experiment will be carried out using AUC weighted as the primary metric with experiment timeout minutes set to 30 minutes and 5 cross-validation folds with the maximum number of iterations that would be executed concurrently set to 4.
All of these settings defines the machine learning task.


The configuration object below contains and persists the parameters for configuring the experiment run, as well as the training data to be used at run time.

In [ ]:
# TODO: Put your automl settings here
automl_settings = {
    "n_cross_validations": 5,
       "primary_metric": 'average_precision_score_weighted',
       "enable_early_stopping": True,
       "experiment_timeout_minutes": 30.0 ,
       "max_concurrent_iterations": 4,
       "max_cores_per_iterations" : 4,
       "enable_voting_ensemble": True,
       "verbosity": logging.INFO
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(
    task = 'classification',
    compute_target =  new_cluster,
    training_data = dataset,
    label_column_name = 'DEATH_EVENT',
    featurization = 'auto',
    blocked_models=['XGBoostClassifier']
)

In [ ]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
from azureml.widgets import RunDetails

RunDetails(remote_run).show()
remote_run.wait_for_completion(show_output=True)

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [ ]:
best_run, fitted_model = automl_run.get_output()
print(best_run)
print(fitted_model)

best_run_metrics = best_run.get_metrics()
for metric_name in best_run_metrics:
    metric = best_run_metrics[metric_name]
    print(metric_name)
    print(metric)

In [ ]:
#TODO: Save the best model

best_run.register_model(model_name='automl_model',model_path='/outputs',properties={'average_precision_score_weighted':best_run_metrics['average_precision_score_weighted']},tags={'Training context':'Auto ML'})

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:
# 

from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies


environment = Environment('capstone-project-environment')
environment.python.conda_dependencies = CondaDependencies.create(pip_packages=[
    'azureml-defaults',
    'inference-schema[numpy-support]',
    'joblib',
    'numpy',
    'scikit-learn=={}'.format(sklearn.__version__)
])

In [ ]:
with open('score.py') as f:
    print(f.read())

In [ ]:
# Create an inference config

from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice


service_name = 'my-custom-env-service'

inference_config = InferenceConfig(entry_script='score.py', environment=environment)
aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1, description="Predict death event using classification model", enable_app_insights=True)

service = Model.deploy(workspace=ws,
                       name=service_name,
                       models=[model],
                       inference_config=inference_config,
                       deployment_config=aci_config,
                       overwrite=True)
service.wait_for_deployment(show_output=True)

In [ ]:
aci_service_name = "aci-service-appinsights"

aci_service = Model.deploy(ws, aci_service_name, [model], inference_config, aci_deployment_config, overwrite=True)
aci_service.wait_for_deployment(show_output=True)

print(aci_service.state)

In [ ]:
if aci_service.state == "Healthy":
    test_sample = json.dumps({
        "data": [
            [1,28,13,45,54,6,57,8,8,10],
            [101,9,8,37,6,45,4,3,2,41]
        ]
    })

    prediction = aci_service.run(test_sample)

    print(prediction)
else:
    raise ValueError("Service deployment isn't healthy, can't call the service. Error: ", aci_service.error)

TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:
input_payload = json.dumps({
    'data': dataset_x[0:2].tolist()
})

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
# Print the logs of the web service

output = service.run(input_payload)

print(output)

# Delete the service
service.delete()